In [78]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [60]:
db_path = 'concerts.db'
conn = sqlite3.connect(db_path)

In [62]:
db_table_names = ('artist', 'customer', 'genre', 'performance', 'ticket', 'ticketpricing')
table_df_dict = dict()

for table_name in db_table_names:
    table_df_dict[table_name] = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)

In [63]:
df = pd.DataFrame()

In [64]:
def populate_with_data(df):
    df['Ticket Price'] = table_df_dict['ticket']['ticketPrice']
    df['Ticket Category'] = table_df_dict['ticket']['type']

    merged_df = pd.merge(table_df_dict['artist'], table_df_dict['genre'], on='idGenre', how='left')
    merged_df = pd.merge(table_df_dict['performance'], merged_df, on='idArtist', how='left')
    merged_df = pd.merge(table_df_dict['ticket'], merged_df, on='idPerformance', how='left')

    df['Artist'] = merged_df['name_x']
    df['Genre'] = merged_df['genreName']
    df['Performance Name'] = merged_df['name_y']
    df['Performance Date'] = merged_df['date']

    df['Official Ticket Price'] = table_df_dict['ticket'].merge(
        table_df_dict['ticketpricing'][['idPerformance', 'Type', 'price']],
        left_on=['idPerformance', 'type'],
        right_on=['idPerformance', 'Type'],
        how='left'
    )['price']

In [65]:
populate_with_data(df)

In [77]:
df.head()

,Ticket Price,Ticket Category,Artist,Genre,Performance Name,Performance Date,Official Ticket Price
0,2000,A,ABX,Pop,ABBA,2025-06-18 20:30:00,2000
1,900,C,Adelaire,Pop,Adele,2025-06-19 20:30:00,900
2,200,D,BamaBeats,Country,Alabama,2025-06-20 20:30:00,200
3,1000,B,ChillSync,Rock,Coldplay,2025-06-21 20:30:00,1000
4,900,A,Distorto,Metal,Disturbed,2025-06-25 21:30:00,900


In [71]:
X_train = df.drop('Ticket Price', axis=1).iloc[0:24]
X_test = df.drop('Ticket Price', axis=1).iloc[24:30]
y_train = df['Ticket Price'].iloc[0:24]
y_test = df['Ticket Price'].iloc[24:30]